<a href="https://colab.research.google.com/drive/1uzH-ybgKrDjNPEvrfwpE47IpD8jKRfnM?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>

###	🔢 What is Multi-index?

Multi-index is a RAG technique that involves using multiple separate indexes or databases to store and retrieve information. Instead of having a single, monolithic index, data is distributed across several specialized indexes. Each index can be optimized for different types of data or query patterns, allowing for more efficient and targeted.

### 🔧 Multi-index RAG Implementation:

1. **Retrieval from Multiple Indices:** We retrieve relevant documents from each vector store.
2. **Context Combination:** We combine the retrieved documents from all sources into a single context.
3. **Response Generation:** Using the combined context, we generate a final response to the original query.
4. **Source Usage Explanation:** We generate an explanation of how information from different sources was used to answer the question.

### ⚙️ Setup

1. **[LLM](https://groq.com/):** Groq's free Open source LLM endpoints([Groq API Key](https://console.groq.com/keys))
2. **[Vector Store](https://www.pinecone.io/learn/vector-database/):** [ChromaDB](https://www.trychroma.com/)
3. **[Embedding Model](https://qdrant.tech/articles/what-are-embeddings/):** [nomic-embed-text-v1.5](https://www.nomic.ai/blog/posts/nomic-embed-text-v1)
4. **[LLM Framework](https://python.langchain.com/v0.2/docs/introduction/):** LangChain
5. **[Huggingface API Key](https://huggingface.co/settings/tokens)**

# Install required libraries

In [11]:
!pip install -q -U \
     Sentence-transformers==3.0.1 \
     langchain==0.3.19 \
     langchain-groq==0.2.4 \
     langchain-chroma==0.2.2 \
     langchain-community==0.3.18 \
     langchain-huggingface==0.1.2 \
     einops==0.8.1

In [ ]:
# 1. Make sure NumPy and dependencies are consistent
!pip install --force-reinstall --no-cache-dir numpy==1.26.4

# 2. Rebuild dependent binary packages that might link to old NumPy
!pip install --force-reinstall --no-cache-dir torch scikit-learn scipy

# 3. (Optional, if using FAISS or Chroma vector stores)
!pip install --force-reinstall --no-cache-dir faiss-cpu chromadb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 91.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 191.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.38.0, but you have google-auth 2.43.0 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
gradio 5.49.1 requires pillow<12.0,>=8.0, but you have pillow 12.0.0 which is incompatible.
gradio 5.49.1 requires pydantic<2.12,>=2.0, 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 179.5 MB/s eta 0:00:00
^C


### Import related libraries related to Langchain, HuggingfaceEmbedding

In [1]:
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain.document_loaders import WebBaseLoader

In [2]:
import os
import getpass

#### Provide a Groq API key. You can create one to access free open-source models at the following link.

[Groq API Creation Link](https://console.groq.com/keys)




In [3]:
os.environ["GROQ_API_KEY"] = getpass.getpass()

··········


# Provide Huggingface API Key. You can create Huggingface API key at following link

[Huggingface API Creation Link](https://huggingface.co/settings/tokens)




In [4]:
os.environ["HF_TOKEN"] = getpass.getpass()

··········


### Step 1: Load and preprocess data code

In [5]:
def load_and_process_data(url):
    # Load data from web
    loader = WebBaseLoader(url)
    data = loader.load()

    # Split text into chunks (Experiment with Chunk Size and Chunk Overlap to get optimal chunking)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = text_splitter.split_documents(data)

    return chunks

### Step 2: Create multiple vector stores code

In [6]:
def create_vector_stores(chunks_list):
    embeddings = HuggingFaceEmbeddings(model_name="nomic-ai/nomic-embed-text-v1.5", model_kwargs = {'trust_remote_code': True})
    vector_stores = []
    for chunks in chunks_list:
        vector_store = Chroma.from_documents(chunks, embeddings)
        vector_stores.append(vector_store)
    return vector_stores

### Step 3: Mult-index-RAG related code

1. **Retrieval from Multiple Indices:** We retrieve relevant documents from each vector store.
2. **Context Combination:** We combine the retrieved documents from all sources into a single context.
3. **Response Generation:** Using the combined context, we generate a final response to the original query.
4. **Source Usage Explanation:** We generate an explanation of how information from different sources was used to answer the question.

In [7]:
def multi_index_rag(query, vector_stores, llm):
    # Retrieve documents from each vector store
    all_docs = []
    for i, vector_store in enumerate(vector_stores):
        docs = vector_store.similarity_search(query, k=3)  # Increased k to 3
        all_docs.extend([f"Source {i+1}: " + doc.page_content for doc in docs])

    # Combine retrieved documents
    context = "\n\n".join(all_docs)

    # Generate response using combined context
    response_prompt = ChatPromptTemplate.from_template(
        "You are an AI assistant tasked with answering questions based on the provided context. "
        "The context contains information from multiple sources related to AI, machine learning, and NLP. "
        "Please analyze the context carefully and provide a comprehensive answer to the question. "
        "If the context doesn't contain enough information, use your general knowledge to supplement the answer, "
        "but prioritize information from the context when available.\n\n"
        "Context:\n{context}\n\n"
        "Question: {query}\n"
        "Answer:"
    )
    response_chain = response_prompt | llm
    try:
        response = response_chain.invoke({"context": context, "query": query})
        final_answer = response.content
    except Exception as e:
        print(f"Error generating response: {e}")
        final_answer = "I apologize, but I encountered an error while generating the response."

    # Generate explanation of source usage
    explanation_prompt = ChatPromptTemplate.from_template(
        "Based on the answer you provided, explain how information from different sources was used to answer the question. "
        "If general knowledge was used to supplement the answer, mention that as well.\n\n"
        "Context: {context}\n"
        "Question: {query}\n"
        "Answer: {answer}\n"
        "Explanation:"
    )
    explanation_chain = explanation_prompt | llm
    try:
        explanation = explanation_chain.invoke({"context": context, "query": query, "answer": final_answer})
        source_explanation = explanation.content
    except Exception as e:
        print(f"Error generating explanation: {e}")
        source_explanation = "Unable to generate explanation due to an error."

    return {
        "final_answer": final_answer,
        "source_explanation": source_explanation,
        "retrieved_context": context
    }

In [11]:
!pip uninstall -y numpy

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4


In [12]:
!pip install --no-cache-dir numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 386.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-chroma 0.2.2 requires numpy<2.0.0,>=1.26.2; python_version >= "3.12", but you have numpy 2.3.4 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.3.4 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.3.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have 

In [11]:
!pip install --force-reinstall --no-cache-dir chromadb llama-index faiss-cpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 316.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 344.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 312.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 188.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 297.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 269.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.1/75.1 kB 242.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-clo

### Step 4: Create chunk of web data to Chroma Vector Store

In [10]:
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.5
)

# Load and process data
urls = [
        "https://en.wikipedia.org/wiki/Artificial_intelligence",
        "https://en.wikipedia.org/wiki/Machine_learning",
        "https://en.wikipedia.org/wiki/Natural_language_processing"
      ]
chunks_list = [load_and_process_data(url) for url in urls]

# Create multiple vector stores
vector_stores = create_vector_stores(chunks_list)

### Step 5: Run Multi-index RAG

This implementation shows the key parts of Multi-index RAG:

1. Use of multiple vector stores representing different sources or types of information
2. Retrieval and combination of information from multiple sources
3. Generation of a comprehensive response using the combined information
Explanation of how different sources contributed to the answer

In [11]:
# Example query
query = "How do AI, machine learning, and NLP relate to each other in modern applications?"

# Run Multi-index RAG
result = multi_index_rag(query, vector_stores, llm)

print("Query:", query)
print("\nFinal Answer:")
print(result["final_answer"])
print("\nSource Usage Explanation:")
print(result["source_explanation"])

Query: How do AI, machine learning, and NLP relate to each other in modern applications?

Final Answer:
Based on the provided context, it is evident that AI, machine learning, and NLP are interconnected concepts that have been a part of AI from the beginning. Here's a comprehensive explanation of their relationship in modern applications:

1. **Machine Learning as a subset of AI**: Machine learning is the study of programs that can improve their performance on a given task automatically (Source 1, Source 2, and Source 3). This means that machine learning is a key component of AI, enabling programs to learn and improve over time.

2. **NLP as a technology powering Document AI**: A Document AI platform sits on top of NLP technology, enabling users to train a computer to extract specific data from different document types (Source 1, Source 2, and Source 3). This demonstrates how NLP is used to power modern AI applications, such as Document AI.

3. **NLP as an enabler of non-technical team